In [29]:
source("source/RaceID3_StemID2_class.R")

In [30]:
x <- read.csv("source/E725.matrix.Seb_NewData_E725.3.quantif",sep="\t",header=TRUE, row.name=1)
prdata <- x[grep("ERCC",rownames(x),invert=TRUE),]

In [3]:
dim(x)
dim(prdata)

[1] 49671   192

[1] 49671   192

In [4]:
require(biomaRt)
# Annotate the data before adding SC experiment

mart <- useMart(biomart = "ENSEMBL_MART_ENSEMBL", dataset = "mmusculus_gene_ensembl")
k <- getBM(
    filters = "ensembl_gene_id",
    attributes = c(
            "ensembl_gene_id",              # Gene stable ID
            "external_gene_name",           # Casual name
            "external_transcript_name",     # Transcript-specific name
            "gene_biotype",                 # Gene biotype
            "transcript_biotype",           # Trans type
            "description",                  # Gene description
            "band",                         # Karyotype band
            "refseq_mrna",
            "go_id",                        # Go Term accession (cellular domains)
            "go_linkage_type",              # Go Term evidence code
            "name_1006",                    # Go Term name
            "definition_1006",              # Go Term definition
            "namespace_1003"                # Go domain                  
    ),
    values = rownames(prdata),
    mart = mart
)

Loading required package: biomaRt


In [39]:
test <- merge(rownames(prdata), k, by.x = 1, by.y = 1)
head(test)

x,external_gene_name,external_transcript_name,gene_biotype,transcript_biotype,description,band,refseq_mrna,go_id,go_linkage_type,name_1006,definition_1006,namespace_1003
ENSMUSG00000000001,Gnai3,Gnai3-201,protein_coding,protein_coding,"guanine nucleotide binding protein (G protein), alpha inhibiting 3 [Source:MGI Symbol;Acc:MGI:95773]",F2.3,NM_010306,GO:0016020,IEA,membrane,"A lipid bilayer along with all the proteins and protein complexes embedded in it an attached to it. [GOC:dos, GOC:mah, ISBN:0815316194]",cellular_component
ENSMUSG00000000001,Gnai3,Gnai3-201,protein_coding,protein_coding,"guanine nucleotide binding protein (G protein), alpha inhibiting 3 [Source:MGI Symbol;Acc:MGI:95773]",F2.3,NM_010306,GO:0046872,IEA,metal ion binding,Interacting selectively and non-covalently with any metal ion. [GOC:ai],molecular_function
ENSMUSG00000000001,Gnai3,Gnai3-201,protein_coding,protein_coding,"guanine nucleotide binding protein (G protein), alpha inhibiting 3 [Source:MGI Symbol;Acc:MGI:95773]",F2.3,NM_010306,GO:0005886,IEA,plasma membrane,The membrane surrounding a cell that separates the cell from its external environment. It consists of a phospholipid bilayer and associated proteins. [ISBN:0716731363],cellular_component
ENSMUSG00000000001,Gnai3,Gnai3-201,protein_coding,protein_coding,"guanine nucleotide binding protein (G protein), alpha inhibiting 3 [Source:MGI Symbol;Acc:MGI:95773]",F2.3,NM_010306,GO:0005886,ISS,plasma membrane,The membrane surrounding a cell that separates the cell from its external environment. It consists of a phospholipid bilayer and associated proteins. [ISBN:0716731363],cellular_component
ENSMUSG00000000001,Gnai3,Gnai3-201,protein_coding,protein_coding,"guanine nucleotide binding protein (G protein), alpha inhibiting 3 [Source:MGI Symbol;Acc:MGI:95773]",F2.3,NM_010306,GO:0005634,ISO,nucleus,"A membrane-bounded organelle of eukaryotic cells in which chromosomes are housed and replicated. In most cells, the nucleus contains all of the cell's chromosomes except the organellar chromosomes, and is the site of RNA synthesis and processing. In some species, or in specialized cell types, RNA metabolism or DNA replication may be absent. [GOC:go_curators]",cellular_component
ENSMUSG00000000001,Gnai3,Gnai3-201,protein_coding,protein_coding,"guanine nucleotide binding protein (G protein), alpha inhibiting 3 [Source:MGI Symbol;Acc:MGI:95773]",F2.3,NM_010306,GO:0005794,IDA,Golgi apparatus,"A compound membranous cytoplasmic organelle of eukaryotic cells, consisting of flattened, ribosome-free vesicles arranged in a more or less regular stack. The Golgi apparatus differs from the endoplasmic reticulum in often having slightly thicker membranes, appearing in sections as a characteristic shallow semicircle so that the convex side (cis or entry face) abuts the endoplasmic reticulum, secretory vesicles emerging from the concave side (trans or exit face). In vertebrate cells there is usually one such organelle, while in invertebrates and plants, where they are known usually as dictyosomes, there may be several scattered in the cytoplasm. The Golgi apparatus processes proteins produced on the ribosomes of the rough endoplasmic reticulum; such processing includes modification of the core oligosaccharides of glycoproteins, and the sorting and packaging of proteins for transport to a variety of cellular locations. Three different regions of the Golgi are now recognized both in terms of structure and function: cis, in the vicinity of the cis face, trans, in the vicinity of the trans face, and medial, lying between the cis and trans regions. [ISBN:0198506732]",cellular_component


In [ ]:
head(lookup_table)


In [13]:
# Update rownames with external_gene_names
matrix_names <- rownames(prdata)
lookup_table <- unique(k[,c(1,3)]) # ensembl, external transcript

new_names <- c()
for (name in matrix_names){
    gene_name <- lookup_table[lookup_table$ensembl_gene_id == name,]$external_transcript_name
    # Not using external_gene_name due to isoforms in data
    if (length(gene_name) != 0){
        new_names <- c(new_names, gene_name)
    } else {
        new_names <- c(new_names, name)
    }
}

In [31]:
dim(lookup_table)
length(rownames(prdata))
length(new_names)

loki <- unique(k[,c(1,3)])
names_only <- unique(loki$ensembl_gene_id)



head(
    merge(names_only, loki, by.x = 1, by.y = 2, all.x = T)
)



#rownames(prdata) <- new_names
#sc <- SCseq(prdata)
#head(unique(k[,c(1,3)]))


[1] 129235      2

[1] 49671

[1] 129396

[1] "ENSMUSG00000000001" "ENSMUSG00000000003" "ENSMUSG00000000028"
[4] "ENSMUSG00000000031" "ENSMUSG00000000037" "ENSMUSG00000000049"

In [ ]:
head(rownames(sc@expdata),100)


In [ ]:
# The data comes from the same batch, so we do not forsee any batch
# effects, therefore downsampling will not be used
sc <- filterdata(
    sc, mintotal=3000, minexpr=5, maxexpr=500, 
    downsample = F, sfn = F, hkn = F,
    dsn = 1, rseed = 17000, CGenes = NULL, FGenes = NULL
)
# Using defaults, we are left with 1000 genes
#dim(sc_defaults@fdata)


In [ ]:
# regress out the batch effect
# optional:
#vars <- data.frame(row.names=names(sc@fdata),batch=sub("(_|)\\d.+","",names(sc@fdata)))
#sc@fdata <- varRegression(sc@fdata,vars)

# correct for cell cycle, proliferation, and expression of degradation markers by PCA
# optional:

gCC <- name2id( k$external_gene_name[k$name_1006 == "cell cycle"],rownames(sc@fdata))
gCP <- name2id( k$external_gene_name[k$name_1006 == "cell proliferation"],rownames(sc@fdata))
vset <- list(gCC,gCP)
#g   <- sub("__chr.+","",rownames(sc@fdata));
#k   <- getBM(attributes = c("external_gene_name", "go_id","name_1006"),filters="external_gene_name",values=g,mart=mart)
#gCC <- name2id( k$external_gene_name[k$name_1006 == "cell cycle"],rownames(sc@fdata)) 
#gCP <- name2id( k$external_gene_name[k$name_1006 == "cell proliferation"],rownames(sc@fdata))
#vset <- list(gCC,gCP)

In [ ]:
k$external_gene_name[k$name_1006 == "cell cycle"]
dim(k)
dim(sc@fdata)
#x <- CCcorrect(sc@fdata,vset=vset,CGenes=NULL,ccor=.4,nComp=NULL,pvalue=.05,quant=.01,mode="pca")
#x$n


In [ ]:
# loadings of the first principal component that has been removed
y <- x$pca$rotation[,x$n[1]]
# genes from vset are either enriched in the head or the tail of this list
tail(y[order(y,decreasing=TRUE)],10)
# reassign the corrected expression matrix to sc@fdata
sc@fdata <- x$xcor

In [ ]:
# k-medoids clustering
sc <- clustexp(sc,clustnr=30,bootnr=50,metric="pearson",do.gap=T,sat=TRUE,SE.method="Tibs2001SEmax",SE.factor=.25,B.gap=50,cln=0,rseed=17000,FUNcluster="kmedoids",FSelect=TRUE)
# compute t-SNE map
sc <- comptsne(sc,rseed=15555,sammonmap=FALSE,initial_cmd=TRUE,fast=TRUE,perplexity=30)
# detect outliers and redefine clusters
sc <- findoutliers(sc, outminc=5,outlg=2,probthr=1e-3,thr=2**-(1:40),outdistquant=.95)
# reassign clusters based on random forest
sc <- rfcorrect(sc,rfseed=12345,final=TRUE,nbfactor=5)

In [ ]:
plotgap(sc)

plotsaturation(sc,disp=TRUE)
# plot change of the within-cluster dispersion as a function of the cluster number: only if sat == TRUE
plotsaturation(sc)
# silhouette of k-medoids clusters
plotsilhouette(sc)
# Jaccard's similarity of k-medoids clusters
plotjaccard(sc)
# barchart of outlier probabilities
plotoutlierprobs(sc)
# regression of background model
plotbackground(sc)
# dependence of outlier number on probability threshold (probthr)
plotsensitivity(sc)
# heatmap of k-medoids cluster
clustheatmap(sc,final=FALSE,hmethod="single")
# heatmap of final cluster
clustheatmap(sc,final=TRUE,hmethod="single")
# highlight k-medoids clusters in t-SNE map
plottsne(sc,final=FALSE)
# highlight final clusters in t-SNE map
plottsne(sc,final=TRUE)